# Introduction to Collective Dynamics on Curves in R³

This notebook provides an interactive introduction to the collective dynamics of particles constrained to move on parametric curves embedded in 3D space.

## Topics Covered:
1. Differential geometry of curves (curvature, torsion, Frenet-Serret frame)
2. Single particle dynamics
3. Multi-particle interactions
4. How curvature affects collective behavior

Author: Collective Dynamics Project

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from src.geometry.curves import helix, circle, viviani_curve, ParametricCurve
from src.dynamics.curve_dynamics import (
    FreeParticleDynamics,
    ConstantSpeedDynamics,
    CurvatureDrivenDynamics,
    MultiParticleSystem,
    kuramoto_interaction,
    attractive_interaction,
    repulsive_interaction
)
from src.visualization.plot_curves import (
    plot_curve_3d,
    plot_curvature_profile,
    plot_trajectory_on_curve,
    plot_multi_particle_trajectories,
    plot_phase_space
)

# For interactive plots
%matplotlib widget
# If widget doesn't work, use: %matplotlib inline

print("Setup complete!")

## Part 1: Differential Geometry of Curves

Let's start by exploring the geometric properties of parametric curves.

A parametric curve in R³ is defined as:
$$\gamma(t) = (x(t), y(t), z(t))$$

Key geometric quantities:
- **Tangent vector**: $T = \frac{\gamma'}{|\gamma'|}$
- **Curvature**: $\kappa = \frac{|\gamma' \times \gamma''|}{|\gamma'|^3}$
- **Torsion**: $\tau = \frac{(\gamma' \times \gamma'') \cdot \gamma'''}{|\gamma' \times \gamma''|^2}$

In [ ]:
# Create a helix and visualize it
curve = helix(radius=1.5, pitch=2.0)

# Plot the curve with Frenet-Serret frames
fig = plt.figure(figsize=(12, 5))

# Left: Curve with Frenet frames
ax1 = fig.add_subplot(121, projection='3d')
frenet_points = np.linspace(0, 4*np.pi, 8)
plot_curve_3d(curve, (0, 4*np.pi), ax=ax1, 
             show_frenet=True, frenet_points=frenet_points, scale=0.5)
ax1.set_title('Helix with Frenet-Serret Frames\n(Red=Tangent, Green=Normal, Blue=Binormal)')

# Right: Curvature and torsion profiles
ax2 = fig.add_subplot(122)
plot_curvature_profile(curve, (0, 4*np.pi), ax=ax2)

plt.tight_layout()
plt.show()

# Print geometric quantities at a point
t = np.pi
frame = curve.frenet_frame(t)
print(f"\nGeometric properties at t={t:.2f}:")
print(f"Position: {curve(t)}")
print(f"Tangent: {frame.tangent}")
print(f"Curvature κ: {frame.curvature:.4f}")
print(f"Torsion τ: {frame.torsion:.4f}")

### Exercise: Explore different curves

Try changing the parameters or using different curves!

In [ ]:
# Try different curves
# curve = circle(radius=2.0)
# curve = viviani_curve(radius=1.0)
# curve = helix(radius=2.0, pitch=0.5)  # Tight helix

# Your code here


## Part 2: Single Particle Dynamics

Now let's study how a single particle moves on a curve under different dynamics.

### 2.1 Free Particle (constant velocity in parameter space)

In [ ]:
# Create a helix
curve = helix(radius=1.5, pitch=2.0)

# Free particle dynamics
dynamics = FreeParticleDynamics(curve)

# Integrate
times, positions, velocities = dynamics.integrate(
    initial_position=0.0,
    initial_velocity=1.0,
    t_span=(0, 15),
    n_points=500
)

# Visualize
fig = plt.figure(figsize=(15, 5))

ax1 = fig.add_subplot(131, projection='3d')
plot_trajectory_on_curve(curve, times, positions, t_range=(0, 6*np.pi), ax=ax1)
ax1.set_title('Free Particle on Helix')

ax2 = fig.add_subplot(132)
ax2.plot(times, positions, 'b-', linewidth=2)
ax2.set_xlabel('Time τ')
ax2.set_ylabel('Position (parameter t)')
ax2.set_title('Position vs Time')
ax2.grid(True, alpha=0.3)

ax3 = fig.add_subplot(133)
plot_phase_space(times, positions, velocities, ax=ax3)

plt.tight_layout()
plt.show()

### 2.2 Curvature-Driven Dynamics

Now let's make the dynamics depend on the local curvature: $\frac{d^2t}{d\tau^2} = f(\kappa(t))$

This is where geometry really affects the motion!

In [ ]:
# Define a force that depends on curvature
# Particle is attracted to high-curvature regions
def curvature_force(kappa):
    return 2.0 * kappa - 0.1

# Create curvature-driven dynamics
dynamics = CurvatureDrivenDynamics(curve, curvature_force, damping=0.1)

# Integrate
times, positions, velocities = dynamics.integrate(
    initial_position=0.0,
    initial_velocity=0.5,
    t_span=(0, 30),
    n_points=1000
)

# Visualize
fig = plt.figure(figsize=(15, 5))

ax1 = fig.add_subplot(131, projection='3d')
plot_trajectory_on_curve(curve, times, positions, t_range=(0, 6*np.pi), ax=ax1)
ax1.set_title('Curvature-Driven Dynamics')

ax2 = fig.add_subplot(132)
plot_curvature_profile(curve, (0, 6*np.pi), ax=ax2, show_torsion=False)
# Overlay particle positions
ax2.scatter(positions, [curve.curvature(t) for t in positions[:100:10]], 
           c='red', s=20, alpha=0.6, label='Particle positions')
ax2.legend()
ax2.set_title('Curvature Profile with Particle Positions')

ax3 = fig.add_subplot(133)
plot_phase_space(times, positions, velocities, ax=ax3)
ax3.set_title('Phase Space')

plt.tight_layout()
plt.show()

print("Notice how the particle's motion is influenced by the curvature!")

### Exercise: Design your own force function

Try different force functions and see how they affect the dynamics!

In [ ]:
# Example: Repulsion from high curvature
# def my_force(kappa):
#     return -1.0 * kappa + 0.5

# Your code here


## Part 3: Multi-Particle Collective Dynamics

Now for the exciting part: multiple particles interacting on a curve!

### 3.1 Kuramoto Synchronization on a Circle

In [ ]:
# Create a circle
curve = circle(radius=2.0)

# Number of particles
n_particles = 8

# Natural frequencies (different for each particle)
natural_frequencies = np.linspace(0.5, 1.5, n_particles)

# Kuramoto coupling
coupling = 2.0
interaction = kuramoto_interaction(coupling, natural_frequencies)

# Create system
system = MultiParticleSystem(curve, n_particles, interaction)

# Initial conditions
initial_positions = np.linspace(0, 2*np.pi, n_particles, endpoint=False)
initial_velocities = natural_frequencies * 0.1

# Integrate
times, positions, velocities = system.integrate(
    initial_positions,
    initial_velocities,
    t_span=(0, 20),
    n_points=1000,
    damping=0.1
)

# Visualize
fig = plt.figure(figsize=(15, 5))

# Positions over time
ax1 = fig.add_subplot(131)
for i in range(n_particles):
    ax1.plot(times, positions[:, i] % (2*np.pi), linewidth=1.5, alpha=0.8)
ax1.set_xlabel('Time')
ax1.set_ylabel('Position (mod 2π)')
ax1.set_title('Kuramoto Synchronization')
ax1.grid(True, alpha=0.3)

# Order parameter
ax2 = fig.add_subplot(132)
order_param = np.array([abs(np.mean(np.exp(1j * positions[i, :]))) 
                        for i in range(len(times))])
ax2.plot(times, order_param, 'r-', linewidth=2)
ax2.set_xlabel('Time')
ax2.set_ylabel('Order Parameter R')
ax2.set_title('Synchronization Measure')
ax2.set_ylim([0, 1.1])
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0.9, color='gray', linestyle='--', alpha=0.5, label='High sync')
ax2.legend()

# Phase differences
ax3 = fig.add_subplot(133)
phase_diffs = np.diff(positions, axis=1)
for i in range(n_particles-1):
    ax3.plot(times, phase_diffs[:, i], linewidth=1, alpha=0.6)
ax3.set_xlabel('Time')
ax3.set_ylabel('Phase Difference')
ax3.set_title('Phase Differences Between Neighbors')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final order parameter: {order_param[-1]:.3f}")
print(f"Synchronization achieved: {order_param[-1] > 0.9}")

### 3.2 Attractive Interactions: Clustering

In [ ]:
# Create a helix
curve = helix(radius=1.5, pitch=2.0)

# Number of particles
n_particles = 10

# Attractive interaction
coupling = 0.5
interaction = attractive_interaction(coupling)

# Create system
system = MultiParticleSystem(curve, n_particles, interaction)

# Initial conditions (random)
np.random.seed(42)
initial_positions = np.random.uniform(0, 2*np.pi, n_particles)
initial_velocities = np.random.uniform(-0.2, 0.2, n_particles)

# Integrate
times_spatial, trajectories = system.spatial_trajectories(
    initial_positions,
    initial_velocities,
    t_span=(0, 30),
    n_points=1000,
    damping=0.2
)

# Visualize
fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(121, projection='3d')
plot_multi_particle_trajectories(curve, times_spatial, trajectories,
                                t_range=(0, 4*np.pi), ax=ax1)
ax1.set_title('Attractive Interactions → Clustering')

# Measure clustering
times, positions, velocities = system.integrate(
    initial_positions,
    initial_velocities,
    t_span=(0, 30),
    n_points=1000,
    damping=0.2
)

ax2 = fig.add_subplot(122)
mean_distances = []
for t_idx in range(len(times)):
    dists = [abs(positions[t_idx, i] - positions[t_idx, j]) 
             for i in range(n_particles) for j in range(i+1, n_particles)]
    mean_distances.append(np.mean(dists))

ax2.plot(times, mean_distances, 'b-', linewidth=2)
ax2.set_xlabel('Time')
ax2.set_ylabel('Mean Pairwise Distance')
ax2.set_title('Clustering Measure')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Initial mean distance: {mean_distances[0]:.3f}")
print(f"Final mean distance: {mean_distances[-1]:.3f}")
print(f"Clustering factor: {mean_distances[0] / mean_distances[-1]:.2f}x")

### 3.3 Repulsive Interactions: Spreading

In [ ]:
# Create a circle
curve = circle(radius=2.0)

# Number of particles
n_particles = 6

# Repulsive interaction
coupling = 1.0
interaction = repulsive_interaction(coupling, cutoff=0.1)

# Create system
system = MultiParticleSystem(curve, n_particles, interaction)

# Initial conditions (clustered)
initial_positions = np.random.uniform(0, 1, n_particles)  # Start clustered
initial_velocities = np.zeros(n_particles)

# Integrate
times, positions, velocities = system.integrate(
    initial_positions,
    initial_velocities,
    t_span=(0, 20),
    n_points=1000,
    damping=0.3
)

# Visualize on polar plot
fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(121, projection='polar')
time_snapshots = [0, len(times)//3, 2*len(times)//3, -1]
colors = ['red', 'orange', 'blue', 'green']
labels = ['t=0', 't=T/3', 't=2T/3', 't=T']

for idx, (t_idx, color, label) in enumerate(zip(time_snapshots, colors, labels)):
    theta = positions[t_idx, :] % (2*np.pi)
    r = np.ones(n_particles) * (1 + idx * 0.3)
    ax1.scatter(theta, r, s=150, color=color, label=label, zorder=10, edgecolors='black')

ax1.set_ylim([0, 2.5])
ax1.set_title('Repulsive Interactions: Particle Spreading\n(Polar View)', pad=20)
ax1.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))

# Time evolution
ax2 = fig.add_subplot(122)
for i in range(n_particles):
    ax2.plot(times, positions[:, i] % (2*np.pi), linewidth=2, alpha=0.8)
ax2.set_xlabel('Time')
ax2.set_ylabel('Position (mod 2π)')
ax2.set_title('Positions Over Time')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Notice how particles spread evenly around the circle!")

## Part 4: Curvature-Dependent Collective Dynamics

Now let's combine geometry and interactions: particles interact on a curve with non-constant curvature.

This is where things get really interesting!

In [ ]:
# Create Viviani's curve (has non-constant curvature)
curve = viviani_curve(radius=1.0)

# First, let's see the curvature profile
fig, ax = plt.subplots(figsize=(10, 4))
plot_curvature_profile(curve, (0, 2*np.pi), ax=ax)
ax.set_title("Viviani's Curve: Variable Curvature")
plt.show()

print("Notice the varying curvature!")
print("Now let's add particles with Kuramoto-like interactions...")

In [ ]:
# Multi-particle system on Viviani's curve
n_particles = 8

# Kuramoto interaction
natural_frequencies = np.linspace(0.8, 1.2, n_particles)
coupling = 1.5
interaction = kuramoto_interaction(coupling, natural_frequencies)

# Create system
system = MultiParticleSystem(curve, n_particles, interaction)

# Initial conditions
initial_positions = np.linspace(0, 2*np.pi, n_particles, endpoint=False)
initial_velocities = natural_frequencies * 0.1

# Integrate
times_spatial, trajectories = system.spatial_trajectories(
    initial_positions,
    initial_velocities,
    t_span=(0, 30),
    n_points=1000,
    damping=0.1
)

times, positions, velocities = system.integrate(
    initial_positions,
    initial_velocities,
    t_span=(0, 30),
    n_points=1000,
    damping=0.1
)

# Visualize
fig = plt.figure(figsize=(16, 6))

# 3D trajectories
ax1 = fig.add_subplot(131, projection='3d')
plot_multi_particle_trajectories(curve, times_spatial, trajectories,
                                t_range=(0, 2*np.pi), ax=ax1)
ax1.set_title('Collective Dynamics on Viviani\'s Curve')

# Positions colored by local curvature
ax2 = fig.add_subplot(132)
for i in range(n_particles):
    # Color by curvature
    curvatures = [curve.curvature(positions[j, i]) for j in range(len(times))]
    scatter = ax2.scatter(times, positions[:, i], c=curvatures, 
                         cmap='viridis', s=1, alpha=0.5)
ax2.set_xlabel('Time')
ax2.set_ylabel('Position')
ax2.set_title('Positions Over Time (colored by local curvature)')
plt.colorbar(scatter, ax=ax2, label='Curvature κ')

# Order parameter
ax3 = fig.add_subplot(133)
order_param = np.array([abs(np.mean(np.exp(1j * positions[i, :]))) 
                        for i in range(len(times))])
ax3.plot(times, order_param, 'r-', linewidth=2)
ax3.set_xlabel('Time')
ax3.set_ylabel('Order Parameter')
ax3.set_title('Synchronization on Curved Manifold')
ax3.set_ylim([0, 1.1])
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal synchronization: {order_param[-1]:.3f}")
print("The curvature affects how particles interact and synchronize!")

## Conclusion

We've explored:

1. **Differential geometry of curves** - curvature, torsion, Frenet-Serret frames
2. **Single particle dynamics** - how geometry affects individual motion
3. **Collective dynamics** - synchronization, clustering, spreading
4. **Geometry-interaction interplay** - how curvature influences collective behavior

## Next Steps

- Experiment with different curves and interaction functions
- Try combining curvature-driven forces with interactions
- Explore what happens with more particles
- Move to 2D manifolds (surfaces in R³)!

## Questions to Explore

1. How does curvature affect synchronization in Kuramoto models?
2. Can particles cluster preferentially in high/low curvature regions?
3. What new phenomena emerge on surfaces compared to curves?
4. How do topological properties (like being closed vs. open) affect dynamics?

In [ ]:
# Your experiments here!
